In [1]:
# Import necessary libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier # Changed: Import XGBoost Classifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
import os
import cv2 # OpenCV for image loading and processing

print("Finished Importing")

Finished Importing


In [5]:
# --- Configuration for your dataset ---
# Path to dataset: structured as "dataset/A/A1.jpg", "dataset/B/B2.jpg", etc.
# IMPORTANT: REPLACE THIS WITH YOUR ACTUAL DATASET PATH
DATASET_PATH = r"C:\Users\Bex\OneDrive\Documents\GitHub\asl_dataset\asl_dataset"
IMAGE_SIZE = (100, 100) # Images will be resized to 100x100 pixels

# --- Your provided image loading function ---
def load_images_and_labels(dataset_path):
    """
    Loads images from the specified dataset path, resizes them,
    converts them to grayscale, and collects their corresponding labels.

    Args:
        dataset_path (str): The root directory of the dataset,
                            expected to contain subdirectories named after labels.

    Returns:
        tuple: A tuple containing:
            - np.array: A NumPy array of image data (flattened pixels).
            - np.array: A NumPy array of corresponding labels.
    """
    X = [] # To store image data (flattened pixel arrays)
    y = [] # To store labels

    print(f"Loading images from: {dataset_path}")

    # Iterate through each subdirectory (which represents a label/class)
    for label in sorted(os.listdir(dataset_path)):
        label_path = os.path.join(dataset_path, label)

        # Skip if it's not a directory
        if not os.path.isdir(label_path):
            continue

        print(f"Processing label: {label}")

        # Iterate through each image file in the current label directory
        for img_name in os.listdir(label_path):
            img_path = os.path.join(label_path, img_name)

            # Read the image in grayscale
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

            # Check if image was loaded successfully
            if img is None:
                print(f"Warning: Could not load image {img_path}. Skipping.")
                continue

            # Resize the image to the predefined IMAGE_SIZE
            img = cv2.resize(img, IMAGE_SIZE)

            # Flatten the 2D image array into a 1D feature vector
            # Normalize pixel values to 0-1 range for better model performance
            X.append(img.flatten() / 255.0)
            y.append(label)

    print(f"Finished loading images. Total images loaded: {len(X)}")
    return np.array(X), np.array(y)

# --- 1. Load Your ASL Handshape Image Data ---

# Call your provided function to load the data
X_raw, y_raw = load_images_and_labels(DATASET_PATH)

# Check if any data was loaded
if len(X_raw) == 0:
    print("\nERROR: No images were loaded. Please check your DATASET_PATH and ensure")
    print("       your dataset structure is correct (e.g., 'dataset_root/A/img.jpg').")
    print("       Exiting script as no data is available for training.")
    exit() # Exit the script if no data is loaded

# Create a Pandas DataFrame from the loaded data
# Each column represents a pixel (feature)
num_features = IMAGE_SIZE[0] * IMAGE_SIZE[1]
df = pd.DataFrame(X_raw, columns=[f'pixel_{i+1}' for i in range(num_features)])
df['label'] = y_raw

print("\n--- Sample of the loaded dataset (first 5 rows, first 10 pixels) ---")
print(df.head().iloc[:, :10]) # Print only first 10 pixel columns for brevity
print(f"\nTotal samples loaded: {len(df)}")
print(f"Image dimensions used: {IMAGE_SIZE[0]}x{IMAGE_SIZE[1]} pixels")
print(f"Number of features (pixels): {num_features}")
print(f"Unique labels found: {df['label'].nunique()} ({df['label'].unique()})")

# --- Data Characteristics Before Training ---
print("\n--- Data Characteristics Before Training ---")
print("Value counts for each label:")
print(df['label'].value_counts())

print("\nStatistical summary of features (first 10 pixels):")
# Ensure X is defined before calling describe on it
X_temp_for_describe = df.drop('label', axis=1)
print(X_temp_for_describe.iloc[:, :10].describe())


# --- 2. Preprocessing the Data ---

# Separate features (X) and target (y)
X = df.drop('label', axis=1)
y = df['label']

# Encode the categorical labels into numerical format
# XGBoost works with numerical targets (0, 1, 2, ... for classes).
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split the data into training and testing sets
# We'll use 80% for training and 20% for testing.
# stratify=y_encoded ensures that the proportion of labels is the same in train and test sets.
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded)

print(f"\nTraining set size: {len(X_train)} samples")
print(f"Testing set size: {len(X_test)} samples")

# --- 3. Train the XGBoost Model ---

# Initialize the XGBoost Classifier model
# objective: 'multi:softmax' for multi-class classification, returns class labels.
# num_class: Number of unique classes in your target variable.
# eval_metric: Metric used for evaluation during training. 'mlogloss' (multiclass logloss) is common.
# use_label_encoder=False: Suppresses a deprecation warning. XGBoost handles integer labels directly.
# n_jobs=-1: Uses all available CPU cores for parallel processing.
model = XGBClassifier(
    objective='multi:softmax',
    num_class=len(label_encoder.classes_), # Set the number of classes dynamically
    eval_metric='mlogloss',
    use_label_encoder=False, # Recommended to suppress warning for future versions
    random_state=42,
    n_jobs=-1
)

# Train the model using the training data
print("\n--- Training the XGBoost model ---")
try:
    model.fit(X_train, y_train)
    print("Model training complete.")
except Exception as e:
    print(f"An error occurred during model training: {e}")
    print("This might indicate issues with data, memory, or model parameters.")


# --- 4. Make Predictions and Evaluate the Model ---

# Only proceed with prediction and evaluation if the model trained successfully
# Changed check to 'feature_importances_' which is a more general indicator of a fitted tree-based model
if hasattr(model, 'feature_importances_'):
    # Make predictions on the test set
    y_pred_encoded = model.predict(X_test)

    # Decode the numerical predictions back to original labels for better understanding
    y_pred = label_encoder.inverse_transform(y_pred_encoded)
    y_test_original = label_encoder.inverse_transform(y_test)

    # Evaluate the model's performance
    accuracy = accuracy_score(y_test_original, y_pred)
    report = classification_report(y_test_original, y_pred, target_names=label_encoder.classes_)

    print(f"\n--- Model Evaluation ---")
    print(f"Accuracy: {accuracy:.4f}")
    print("\nClassification Report:\n", report)

    # --- Example of making a prediction on a new, unseen image ---
    print("\n--- Example Prediction for a Single New Image ---")

    # To make a prediction on a new image:
    # 1. Load the new image using cv2.imread(new_img_path, cv2.IMREAD_GRAYSCALE)
    # 2. Resize it: cv2.resize(img, IMAGE_SIZE)
    # 3. Flatten it and normalize: img.flatten() / 255.0
    # 4. Reshape it for prediction: new_image_features.reshape(1, -1)

    # For demonstration, let's pick a random image from the test set and "re-process" it
    # as if it were a new, unseen image to show the prediction flow.
    if len(X_test) > 0:
        random_idx = np.random.randint(0, len(X_test))
        sample_image_features = X_test.iloc[random_idx].values.reshape(1, -1)
        true_label = y_test_original[random_idx]

        predicted_label_encoded = model.predict(sample_image_features)
        predicted_letter = label_encoder.inverse_transform(predicted_label_encoded)

        print(f"True ASL letter for sample: {true_label}")
        print(f"Predicted ASL letter: {predicted_letter[0]}")
    else:
        print("Not enough data in the test set to demonstrate a new prediction example.")
else:
    print("\nModel did not train successfully, skipping prediction and evaluation.")


Loading images from: C:\Users\Bex\OneDrive\Documents\GitHub\asl_dataset\asl_dataset
Processing label: a
Processing label: b
Processing label: c
Processing label: d
Processing label: e
Processing label: f
Processing label: g
Processing label: h
Processing label: i
Processing label: j
Processing label: k
Processing label: l
Processing label: m
Processing label: n
Processing label: o
Processing label: p
Processing label: q
Processing label: r
Processing label: s
Processing label: t
Processing label: u
Processing label: v
Processing label: w
Processing label: x
Processing label: y
Processing label: z
Finished loading images. Total images loaded: 1815

--- Sample of the loaded dataset (first 5 rows, first 10 pixels) ---
   pixel_1  pixel_2  pixel_3  pixel_4  pixel_5  pixel_6  pixel_7  pixel_8  \
0      0.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
1      0.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
2      0.0      0.0      0.0      0.0      0

C:\Users\Bex\anaconda3\envs\tf-env\lib\site-packages\xgboost\core.py:158: UserWarning: [21:32:59] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Model training complete.

--- Model Evaluation ---
Accuracy: 0.9477

Classification Report:
               precision    recall  f1-score   support

           a       1.00      1.00      1.00        14
           b       0.93      0.93      0.93        14
           c       0.93      1.00      0.97        14
           d       0.88      1.00      0.93        14
           e       0.93      0.93      0.93        14
           f       0.93      1.00      0.97        14
           g       0.87      0.93      0.90        14
           h       1.00      0.93      0.96        14
           i       0.93      1.00      0.97        14
           j       1.00      1.00      1.00        14
           k       1.00      0.86      0.92        14
           l       1.00      1.00      1.00        14
           m       1.00      0.93      0.96        14
           n       0.93      1.00      0.97        14
           o       1.00      0.93      0.96        14
           p       1.00      0.93      0.9